In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
path_base = '/content/drive/MyDrive/Tesis/SICK/transformer_tunned_BERT/uncase_base/study_optuna/'
file_root = '/content/drive/MyDrive/Tesis/SICK/uncase_based/SICK.txt'
with open(file_root, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split("\t")) for l in temp_f.readlines() ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getSICKSents(filename='sts-train.csv', set_type='TRAIN', encoding='utf-8'):
  f = open(filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for i, line in enumerate(f):
    example = re.split(r'\t+', line)
    if i >= 1:
      if example[-1].strip() == set_type:
        s2.append(example[1])
        s1.append(example[2])
        target.append(float(example[4]))
  print("samples: {}".format(len(target)))
  return s1, s2, target

In [ ]:
s1_train,s2_train,target_train = getSICKSents(file_root, 'TRAIN')

samples: 4439


In [ ]:
s1_test,s2_test,target_test= getSICKSents(file_root, 'TEST')

samples: 4906


In [ ]:
s1_dev,s2_dev,target_dev= getSICKSents(file_root, 'TRIAL')

samples: 495


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [ ]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'SICK'
BERT_PATH = "bert-base-uncased"

In [ ]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [ ]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [ ]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [ ]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order.
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [ ]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 139
Number of evaluation batches: 16


In [ ]:
class BertBaseUncasedRegressor(nn.Module):
  def __init__(self, num_classes = 1, dropout=0.1, attentions=True, n_layers = 0):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      # Return:
      # last_hidden_state.shape -> [batch_size, num_tokens_in_sequence,hidden_size] (bert_base hidden_size = 768)
      # pooler_output.shape -> [batch_size, hidden_size] se utiliza una capa de pooling simple que aplica una transformación
      # lineal seguida de una función de activación tangente hiperbólica (tanh) a la última representación oculta (last_hidden_state) del token [CLS].
      layers = []
      for _ in range(n_layers):
          layers.append(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)) #self.bert.config.hidden_size -> nos da el tamaño oculto
          layers.append(nn.Dropout(dropout))
      layers.append(nn.Linear(self.bert.config.hidden_size, num_classes)) #self.bert.config.hidden_size -> nos da el tamaño oculto

      self.regressor = nn.Sequential(*layers)


  def forward(self, input_ids, type_ids, mask):
      bert_output =self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      output_max_pooling = torch.max(bert_output.last_hidden_state, dim=1)[0]
      #output = bert_output.pooler_output
      #output = bert_output.last_hidden_state[:, 0, :]
      logits = self.regressor(output_max_pooling)
      #logits = self.regressor(output_drop)
      return logits, bert_output

In [ ]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [ ]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [ ]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

    for i, batch in enumerate(loader):

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman


In [ ]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())

        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)

    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [ ]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")

      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()

      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [ ]:
study_name = 'study-bert-max-pooling-base-SICK'

In [ ]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [ ]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 0, 3),
      "dropout": trial.suggest_uniform("dropout", 0, 0.7),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedRegressor(dropout=params["dropout"], n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

  NUM_EPOCHS = 4
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)

  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):

    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [ ]:
storage_path = path_base + "optuna_study_max_pooling_SICK_.db"

In [ ]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=f"sqlite:///{storage_path}", load_if_exists=True)

[I 2023-11-17 18:51:12,751] Using an existing study with name 'study-bert-max-pooling-base-SICK' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=15)

<ipython-input-29-b262b188575f>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-29-b262b188575f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 85.4741s | Train loss: 0.8387 | Dev loss: 0.3717
Train Pearson Coef: 0.5641 | Dev Pearson Coef: 0.8395
Train Spearman Coef: 0.5032 | Dev Spearman Coef: 0.8029
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 84.9133s | Train loss: 0.2900 | Dev loss: 0.3621
Train Pearson Coef: 0.8556 | Dev Pearson Coef: 0.8530
Train Spearman Coef: 0.7831 | Dev Spearman Coef: 0.8096
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 87.2763s | Train loss: 0.1831 | Dev loss: 0.2718
Train Pearson Coef: 0.9109 | Dev Pearson Coef: 0.8585
Train Spearman Coef: 0.8761 | Dev Spearman Coef: 0.8236
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 87.2674s | Train loss: 0.1439 | Dev loss: 0.6836
Train Pearson Coef: 0.9314 | Dev Pearson Coef: 0.8779
Train Spearm

[I 2023-11-17 18:57:29,533] Trial 15 finished with value: 0.27179756201803684 and parameters: {'num_layer': 1, 'dropout': 0.6942507858478799, 'optimizer': 'Adam', 'learning_rate': 3.387276502348421e-05}. Best is trial 14 with value: 0.2511565205641091.
<ipython-input-29-b262b188575f>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-29-b262b188575f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 86.3021s | Train loss: 1.5928 | Dev loss: 1.2777
Train Pearson Coef: 0.0019 | Dev Pearson Coef: 0.3210
Train Spearman Coef: -0.0065 | Dev Spearman Coef: 0.2806
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 86.3905s | Train loss: 0.6026 | Dev loss: 0.6314
Train Pearson Coef: 0.6566 | Dev Pearson Coef: 0.7830
Train Spearman Coef: 0.5082 | Dev Spearman Coef: 0.7357
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 86.3680s | Train loss: 0.3251 | Dev loss: 0.5516
Train Pearson Coef: 0.8327 | Dev Pearson Coef: 0.8269
Train Spearman Coef: 0.7391 | Dev Spearman Coef: 0.7651


...

--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 86.2216s | Train loss: 0.1451 | Dev loss: 0.9231
Train Pearson Coef: 0.9287 | Dev Pearson Coef: 0.8545
Train Spearman Coef: 0.8908 | Dev Spearman Coef: 0.8033
--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 83.0180s | Train loss: 13.1207 | Dev loss: 12.7730
Train Pearson Coef: 0.0274 | Dev Pearson Coef: 0.0925
Train Spearman Coef: 0.0164 | Dev Spearman Coef: 0.0735
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 83.1022s | Train loss: 12.3489 | Dev loss: 12.0404
Train Pearson Coef: 0.0346 | Dev Pearson Coef: 0.1054
Train Spearman Coef: 0.0228 | Dev Spearman Coef: 0.0869
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 83.0358s | Train loss: 11.6032 | Dev loss: 11.3319
Train Pearson Coef: 0.0417 | Dev Pearson Coef: 0.1165
Train 

[I 2023-11-17 20:13:21,676] Trial 28 finished with value: 10.646187976002693 and parameters: {'num_layer': 1, 'dropout': 0.4679257308649918, 'optimizer': 'SGD', 'learning_rate': 4.373140271727492e-05}. Best is trial 25 with value: 0.23423976683989167.


--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 87.2417s | Train loss: 1.6562 | Dev loss: 0.4443
Train Pearson Coef: 0.2866 | Dev Pearson Coef: 0.7952
Train Spearman Coef: 0.3108 | Dev Spearman Coef: 0.7297
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 87.2307s | Train loss: 0.4161 | Dev loss: 0.3792
Train Pearson Coef: 0.7857 | Dev Pearson Coef: 0.8318
Train Spearman Coef: 0.6878 | Dev Spearman Coef: 0.7845
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 87.2888s | Train loss: 0.1971 | Dev loss: 0.3684
Train Pearson Coef: 0.9022 | Dev Pearson Coef: 0.8355
Train Spearman Coef: 0.8520 | Dev Spearman Coef: 0.7898


[I 2023-11-17 20:19:17,458] Trial 29 finished with value: 0.36843062564730644 and parameters: {'num_layer': 2, 'dropout': 0.5747084414730621, 'optimizer': 'Adam', 'learning_rate': 1.0382224269405786e-05}. Best is trial 25 with value: 0.23423976683989167.


--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 87.1860s | Train loss: 0.1179 | Dev loss: 0.4546
Train Pearson Coef: 0.9427 | Dev Pearson Coef: 0.8432
Train Spearman Coef: 0.9166 | Dev Spearman Coef: 0.7987


In [ ]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics)
print(len(df_optuna))
print(len(df_metrics ))

30
30


In [ ]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

Old trials:  134


In [ ]:
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

NameError: ignored

In [ ]:
if len(df_optuna) == len(df_metrics):
  df_complete_updated = pd.concat([df_optuna, df_metrics], axis=1, sort=False)

In [ ]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [ ]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

In [ ]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                 25
value                                                             0.23424
params_dropout                                                     0.5562
params_learning_rate                                             0.000028
params_num_layer                                                        1
params_optimizer                                                  RMSprop
state                                                            COMPLETE
train_loss              [0.7403897465132981, 0.23444120266990695, 0.12...
valid_loss              [0.28351533878594637, 0.39966722298413515, 0.2...
spearman_train          [0.549730489046594, 0.8236727233700721, 0.9141...
spearman_val            [0.8254026565401088, 0.8087718480670142, 0.832...
pearson_train           [0.6145939982373084, 0.8833329554859091, 0.941...
pearson_val             [0.8664709020950482, 0.8658339741718765, 0.879...
Name: 25, dtype: object


In [ ]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
#os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>